# EA Assignment 05 - Topic Modelling
__Authored by: Álvaro Bartolomé del Canto (alvarobartt @ GitHub)__

---

<img src="https://media-exp1.licdn.com/dms/image/C561BAQFjp6F5hjzDhg/company-background_10000/0?e=2159024400&v=beta&t=OfpXJFCHCqdhcTu7Ud-lediwihm0cANad1Kc_8JcMpA">

We will start this Jupyter Notebook with a little recap from the previous one named `02 - Data Preprocessing.ipynb` where we defined the NLP Preprocessing pipeline we want to apply in order to prepare the input raw text into valuable text, at the end of that Jupyter Notebook we dumped the generated `pandas.DataFrame` into a JSON-Lines file so as to avoid preprocessing the data on each Notebook. So on, this Jupyter Notebook starts with the preprocessed data loading, which means that if you want to have more details/insights on how the data is being preprocessed just refer to the previous notebook.

## Loading PreProcessed Data

__Reproducibility Warning__: you will not find the `PreProcessedDocuments.jsonl` file when cloning the repository from GitHub, since it has been included in the .gitignore file due to the GitHub quotas when uploading big files. So on, if you want to reproduce this Jupyter Notebook, please refer to `02 - Data Preprocessing.ipynb` where the NLP preprocessing pipeline is explained and this file is generated.

In [1]:
import json

data = list()

with open('PreProcessedDocuments.jsonl', 'r') as f:
    for line in f.readlines():
        data.append(json.loads(line))

In [2]:
import pandas as pd

data = pd.DataFrame(data)
data.head()

,lang,context,preprocessed_text
0,en,wikipedia,watchmen twelve issue comic book limited serie...
1,en,wikipedia,citigroup center formerly citicorp center tall...
2,en,wikipedia,birth_place death_date death_place party conse...
3,en,wikipedia,marbod maroboduus born died king marcomanni no...
4,en,wikipedia,sylvester medal bronze medal awarded every yea...


In [3]:
data.shape

(23011, 3)

In [4]:
data = data[(data['lang'] == 'en') & (data['context'] == 'wikipedia')]
data.shape

(4000, 3)

In [5]:
data['tokenized_text'] = data['preprocessed_text'].str.split(' ')
data.head()

,lang,context,preprocessed_text,tokenized_text
0,en,wikipedia,watchmen twelve issue comic book limited serie...,"[watchmen, twelve, issue, comic, book, limited..."
1,en,wikipedia,citigroup center formerly citicorp center tall...,"[citigroup, center, formerly, citicorp, center..."
2,en,wikipedia,birth_place death_date death_place party conse...,"[birth_place, death_date, death_place, party, ..."
3,en,wikipedia,marbod maroboduus born died king marcomanni no...,"[marbod, maroboduus, born, died, king, marcoma..."
4,en,wikipedia,sylvester medal bronze medal awarded every yea...,"[sylvester, medal, bronze, medal, awarded, eve..."


---

## Tackling the Topic Modelling

So as to tackle this problem, we will just create a Topic Modelling model for each unique combination of 'context' and 'language', since without a proper Machine Translation model to translate/unify all the text in the dataset to English. It is useless to apply the same Topic Modelling model to data written in different languages since there will no be relation even though between texts of the same topic in the same context.

For example, a couple of Wikipedia's texts of the topic "Historical Events" written in English and French will no have any common words/tokens except the non-specific language words such as names, surnames, etc.

So on, we will be testing some commonly used Topic Modelling algorithms splitting the data by context and language in order to get to know both the number of suitable topics from the main words of each topic and then visualize them in a 2D plot.

---

## Topic Modelling with LDA

Latent Dirichlet Allocation ...


__Note__: since we are just testing the most common Topic Modelling algorithms, we will be using just the Wikipedia texts written in English. 

In [6]:
import gensim

In [7]:
id2word = gensim.corpora.Dictionary(data['tokenized_text'])
list(id2word.token2id.items())[:5]

[('abandon', 0),
 ('abbreviated', 1),
 ('abc', 2),
 ('abilities', 3),
 ('ability', 4)]

In [8]:
id2word.filter_extremes(no_below=10, no_above=0.05)

In [9]:
corpus = [
    id2word.doc2bow(document=text, allow_update=True) for text in data['tokenized_text']
]

In [10]:
%time lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=5, passes=5)

CPU times: user 2min 10s, sys: 4min 24s, total: 6min 35s
Wall time: 45.9 s


In [11]:
from pprint import pprint
pprint(lda.print_topics(num_words=5))

[(0,
  '0.004*"used" + 0.004*"may" + 0.003*"use" + 0.003*"system" + 0.002*"time"'),
 (1,
  '0.005*"city" + 0.003*"war" + 0.003*"century" + 0.002*"world" + '
  '0.002*"state"'),
 (2,
  '0.003*"world" + 0.002*"party" + 0.002*"used" + 0.002*"time" + '
  '0.002*"german"'),
 (3,
  '0.003*"used" + 0.003*"world" + 0.003*"water" + 0.003*"species" + '
  '0.003*"may"'),
 (4,
  '0.004*"film" + 0.004*"album" + 0.003*"music" + 0.003*"band" + 0.003*"time"')]


In [18]:
# lda.save('resources/lda/model')

In [12]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [13]:
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.069481 -0.126207       1        1  32.433822
4      0.108073  0.081870       2        1  24.065001
0     -0.127872  0.021613       3        1  17.814139
3     -0.092058 -0.012808       4        1  14.344605
2      0.042376  0.035531       5        1  11.342434, topic_info=         Term         Freq        Total Category  logprob  loglift
21273   album  4592.000000  4592.000000  Default  30.0000  30.0000
21744    film  5084.000000  5084.000000  Default  29.0000  29.0000
21456    city  9113.000000  9113.000000  Default  28.0000  28.0000
22729    band  3591.000000  3591.000000  Default  27.0000  27.0000
23306   music  4563.000000  4563.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
21778  french   566.482178  3753.811772   Topic5  -6.7640   0.2855
21953   known   605.084864  6407.528201   Topic5  -6.6981  -0.1832
21632   early   582.273295  4677.071870   Topic5  -6.7365   0.0931
21417  called   572.174332  5369.106035   Topic5  -6.7540  -0.0624
22193  people   556.246364  6328.975959   Topic5  -6.7822  -0.2551

[547 rows x 6 columns], token_table=        Topic      Freq        Term
term                               
104427      2  0.010475  abramovich
104427      4  0.974188  abramovich
104427      5  0.010475  abramovich
21252       1  0.031989       actor
21252       2  0.918540       actor
...       ...       ...         ...
22719       3  0.093232        york
22719       4  0.037350        york
22719       5  0.063580        york
125869      4  0.988860    zamorano
172923      3  0.989831      zoster

[1657 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 1, 4, 3])